# Importando bibliotecas

In [1]:
#!pip install i18n
#!pip install dotenv

In [1]:
#!pip install pandas
#!pip install investpy
#!pip install selenium
#!pip install webdriver-manager

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

#!pip install investpy
#import investpy as inv
from time import sleep
import time

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
from selenium import webdriver

# Funcoes criadas

In [4]:
def converter_valor(valor):
    if valor.endswith('B'):
        return float(valor.replace('B', '').replace(' ', '')) * 1_000
    elif valor.endswith('M'):
        return float(valor.replace('M', '').replace(' ', '')) * 1
    elif valor.endswith('K'):
        return float(valor.replace('K', '').replace(' ', '')) * 0.001
    elif valor == 'NA':
        return float('nan')

# Pegando os dados

In [5]:
codigos_ibovespa = [
    'ABEV3', #
    #'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    #'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
   # 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     #'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     #'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    #'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    #'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
   # 'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]

# Fazer itercao dao comeco

In [6]:
refazer_comeco = True
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [ ]:
codigos_ibovespa = [
    'ABEV3', 
    'AZUL4', 
    'B3SA3',
    'BBDC4'
    ]
if refazer_comeco:    
    chrome_driver_path = "/usr/bin/chromedriver"

    service = Service(executable_path=chrome_driver_path)

    options = Options()
    options.add_argument("--headless")  # Executar em modo headless
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
    options.add_argument("--disable-gpu")  # Desativar o uso de GPU
    options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela
    #setor_financeiro = ['BBAS3','BBDC3']
    setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11'}
    lista_dataframes = []
    navegador = webdriver.Chrome(service=service, options=options)
    iteracao=0
    for acao in codigos_ibovespa:
        if acao in setor_financeiro:
            print(f'Execucao da {acao} do setor financeiro.')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')

            dados = {
                'datas' : datas,
                'Receita Líquida' : [],
                'Resultado Bruto' :[],
                'EBIT' : [np.nan] * len(datas),
                'Depreciação e Amortização' : [np.nan] * len(datas),
                'EBITDA' : [np.nan] * len(datas),
                'Lucro Líquido' : [],
                'Lucro/Ação' : [np.nan] * len(datas),}
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)

                metricas = [
                'Receita Líquida',
                'Resultado Bruto',
                'Lucro Líquido',
                ]
                for i in range(0, len(lista_dre_ultimo_mes)):
                    chave = lista_dre_ultimo_mes[i]
                    if chave in metricas:
                        valor = lista_dre_ultimo_mes[i + 1].replace('R$','').replace(',','.').strip()
                        dados[chave].append(valor)

        else:
            print(f'Execucao da {acao}')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
           
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            dados = {
                'datas' : datas,
                'Receita Líquida' : [],
                'Resultado Bruto' :[],
                'EBIT' : [],
                'Depreciação e Amortização' : [],
                'EBITDA' : [],
                'Lucro Líquido' : [],
                'Lucro/Ação' : [],}
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)
                
                metricas = [
                'Receita Líquida',
                'Resultado Bruto',
                'Lucro Líquido',
                'EBIT',
                'Depreciação e Amortização',
                'EBITDA',
                'Lucro/Ação',

                ]
                for i in range(0, len(lista_dre_ultimo_mes)):
                    chave = lista_dre_ultimo_mes[i]
                    if chave in metricas:
                        valor = lista_dre_ultimo_mes[i + 1].replace('R$','').replace(',','.').strip()
                        dados[chave].append(valor)
                
        df_dre_ultimo_mes = pd.DataFrame(dados)
        df_dre_ultimo_mes = df_dre_ultimo_mes.rename(
            columns=lambda coluna: coluna.replace("/", "_").replace(" ", "_").lower())

        df_dre_ultimo_mes['datas'] = pd.to_datetime(df_dre_ultimo_mes['datas'], format='%d/%m/%Y')
        df_dre_ultimo_mes['datas'] = df_dre_ultimo_mes['datas'].dt.strftime('%Y-%m-%d')
        

        if acao in setor_financeiro:
            colunas_selecionadas = df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'datas') & 
                                    (df_dre_ultimo_mes.columns != 'lucro_ação')]
            for col in colunas_selecionadas:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)   
        else:
            for col in df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'datas') & (df_dre_ultimo_mes.columns != 'lucro_ação')]:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)

        df_dre_ultimo_mes['tic'] = acao

        lista_dataframes.append(df_dre_ultimo_mes)

        iteracao = iteracao+1

        if iteracao % 15 == 0:
            pd.concat(lista_dataframes).to_csv('dre_trimestral.csv')

        end_time = time.time()

        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time

        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60

        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

In [29]:
df_resumo_balanco = pd.concat(lista_dataframes)

In [31]:
df_resumo_balanco

,datas,receita_líquida,resultado_bruto,ebit,depreciação_e_amortização,ebitda,lucro_líquido,lucro_ação,tic
0,2024-09-30,22100.00,11120.00,5350.00,-1660.0,7020.0,3460.00,0.22,ABEV3
1,2024-06-30,20040.00,9980.00,4050.00,-1620.0,5660.0,2400.00,0.15,ABEV3
2,2024-03-31,20280.00,10220.00,4880.00,-1570.0,6450.0,3700.00,0.23,ABEV3
3,2023-12-31,19990.00,10690.00,5430.00,-1400.0,6820.0,4390.00,0.28,ABEV3
4,2023-09-30,20320.00,10090.00,4900.00,-1590.0,6480.0,3910.00,0.25,ABEV3
...,...,...,...,...,...,...,...,...,...
61,2009-06-30,378.24,378.24,250.04,NaN,NaN,188.13,0.09,B3SA3
62,2009-03-31,316.55,316.55,167.79,NaN,NaN,226.98,0.11,B3SA3
63,2008-12-31,370.44,370.44,69.50,NaN,NaN,115.39,0.06,B3SA3
64,2008-09-30,404.68,404.68,97.46,NaN,NaN,200.97,0.10,B3SA3


In [12]:
df_resumo_balanco.columns[(df_resumo_balanco.columns != 'data') & (df_resumo_balanco.columns != 'lucro_por_acao') & (df_resumo_balanco.columns != 'tic')]

Index(['receita_liquida', 'resultado_bruto', 'ebit', 'depreciacao_amortizacao',
       'ebitida', 'lucro_liquido'],
      dtype='object')

# Fazer da onde parou

In [9]:
data_frame_feito = pd.read_csv('dre_trimestral_2.csv').iloc[:,1:]

In [16]:
data_frame_feito.tic.unique()

array(['ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3',
       'BEEF3', 'BPAC11', 'BRAP4', 'BRFS3', 'BRKM5', 'CCRO3', 'CIEL3',
       'CMIG4', 'COGN3', 'CPFE3', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3',
       'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENGI11',
       'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
       'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
       'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3',
       'PETR4', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RENT3', 'SANB11',
       'SBSP3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5'],
      dtype=object)

In [ ]:
lista_dataframes = []
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11','ITUB4','SANB11','IRBR3'}
iteracao=0
for acao in codigos_ibovespa:
    if acao in data_frame_feito['tic'].unique():
        lista_dataframes.append(data_frame_feito[data_frame_feito.tic == acao])
        print(f'Essa {acao} ja foi')
    else:
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
        # Iniciar o navegador com o serviço
        navegador = webdriver.Chrome(service=servico)
      
        
        if acao in setor_financeiro:
            print(f'Execucao da {acao} do setor financeiro.')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_12meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)


                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebit.append(np.nan)
                depreciacao_amortizacao.append(np.nan)
                lista_ebitda.append(np.nan)
                lucro_acao.append(np.nan)
        elif acao in ['IRBR3']:
            
            print(f'Execucao da {acao}')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_12meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)
                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lista_ebit.append(lista_dre_ultimo_mes[5].replace('R$','').replace(',','.').strip())
                depreciacao_amortizacao.append(np.nan)
                lista_ebitda.append(np.nan)
                lucro_liquido.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lucro_acao.append(np.nan)

        else:
            print(f'Execucao da {acao}')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_12meses"]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)
                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lista_ebit.append(lista_dre_ultimo_mes[5].replace('R$','').replace(',','.').strip())
                depreciacao_amortizacao.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebitda.append(lista_dre_ultimo_mes[9].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[11].replace('R$','').replace(',','.').strip())
                lucro_acao.append(lista_dre_ultimo_mes[13].replace('R$','').replace(',','.').strip())

        listas_dados = {
                'data':datas,
                'receita_liquida': receita_liquida,
                'resultado_bruto': resultado_bruto,
                'ebit': lista_ebit,
                'depreciacao_amortizacao': depreciacao_amortizacao,
                'ebitida': lista_ebitda,
                'lucro_liquido': lucro_liquido,
                'lucro_por_acao': lucro_acao
            }
        df_dre_ultimo_mes = pd.DataFrame(listas_dados)
        df_dre_ultimo_mes['data'] = pd.to_datetime(df_dre_ultimo_mes['data'], format='%d/%m/%Y')
        df_dre_ultimo_mes['data'] = df_dre_ultimo_mes['data'].dt.strftime('%Y-%m-%d')

        if acao in setor_financeiro:
            colunas_selecionadas = df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & 
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                        (df_dre_ultimo_mes.columns != 'ebit') &   
                                        (df_dre_ultimo_mes.columns != 'depreciacao_amortizacao') & 
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                        (df_dre_ultimo_mes.columns != 'ebitida') &  
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao')]
            for col in colunas_selecionadas:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)  

        elif acao in ['IRBR3']:
            for col in df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns == 'receita_liquida') | (df_dre_ultimo_mes.columns == 'resultado_bruto')|
                                                (df_dre_ultimo_mes.columns == 'ebit') | (df_dre_ultimo_mes.columns == 'lucro_liquido')]:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)
            
        else:
            for col in df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & (df_dre_ultimo_mes.columns != 'lucro_por_acao')]:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)

        df_dre_ultimo_mes['tic'] = acao

        lista_dataframes.append(df_dre_ultimo_mes)

        iteracao = iteracao+1

        if iteracao % 5 == 0:
            pd.concat(lista_dataframes).to_csv('dre_trimestral.csv')

        end_time = time.time()

        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time

        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60

        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")   

In [20]:
pd.concat(lista_dataframes).tic.unique()

70

In [21]:
pd.concat(lista_dataframes).to_csv('dre_trimestral.csv')

In [ ]:
lista_dataframes = []
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11'}
iteracao=0
for acao in codigos_ibovespa:
    if acao in data_frame_feito['tic'].unique():
        lista_dataframes.append(data_frame_feito[data_frame_feito.tic == acao])
        print(f'Essa {acao} ja foi')
    else:
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
        # Iniciar o navegador com o serviço
        navegador = webdriver.Chrome(service=servico)
      
        
        if acao in setor_financeiro:
            print(f'Execucao da {acao} do setor financeiro.')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)


                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebit.append(np.nan)
                depreciacao_amortizacao.append(np.nan)
                lista_ebitda.append(np.nan)
                lucro_acao.append(np.nan)

        else:
            print(f'Execucao da {acao}')
            start_time = time.time() 
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            #tabela = navegador.find_element(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]')

            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]

            # Encontrar a tabela com o id
            tabela = navegador.find_element(By.ID, 'tabela_resumo_empresa_dre_3meses')
            receita_liquida = []
            resultado_bruto  = []
            lista_ebit = []
            depreciacao_amortizacao = []
            lista_ebitda = []
            lucro_liquido = []
            lucro_acao = []
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select')#/html/body/div[1]/div[4]/div[2]/div[1]/div[3]/table[2]/thead/tr/th[2]/select
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                # Iterar pelas linhas e capturar os dados
                lista_dre_ultimo_mes = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_dre_ultimo_mes.append(celula.text)
                receita_liquida.append(lista_dre_ultimo_mes[1].replace('R$','').replace(',','.').strip())
                resultado_bruto.append(lista_dre_ultimo_mes[3].replace('R$','').replace(',','.').strip())
                lista_ebit.append(lista_dre_ultimo_mes[5].replace('R$','').replace(',','.').strip())
                depreciacao_amortizacao.append(lista_dre_ultimo_mes[7].replace('R$','').replace(',','.').strip())
                lista_ebitda.append(lista_dre_ultimo_mes[9].replace('R$','').replace(',','.').strip())
                lucro_liquido.append(lista_dre_ultimo_mes[11].replace('R$','').replace(',','.').strip())
                lucro_acao.append(lista_dre_ultimo_mes[13].replace('R$','').replace(',','.').strip())

        listas_dados = {
                'data':datas,
                'receita_liquida': receita_liquida,
                'resultado_bruto': resultado_bruto,
                'ebit': lista_ebit,
                'depreciacao_amortizacao': depreciacao_amortizacao,
                'ebitida': lista_ebitda,
                'lucro_liquido': lucro_liquido,
                'lucro_por_acao': lucro_acao
            }
        df_dre_ultimo_mes = pd.DataFrame(listas_dados)
        df_dre_ultimo_mes['data'] = pd.to_datetime(df_dre_ultimo_mes['data'], format='%d/%m/%Y')
        df_dre_ultimo_mes['data'] = df_dre_ultimo_mes['data'].dt.strftime('%Y-%m-%d')

        if acao in setor_financeiro:
            colunas_selecionadas = df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & 
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                        (df_dre_ultimo_mes.columns != 'ebit') &   
                                        (df_dre_ultimo_mes.columns != 'depreciacao_amortizacao') & 
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao') &
                                        (df_dre_ultimo_mes.columns != 'ebitida') &  
                                        (df_dre_ultimo_mes.columns != 'lucro_por_acao')]
            for col in colunas_selecionadas:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)   
        else:
            for col in df_dre_ultimo_mes.columns[(df_dre_ultimo_mes.columns != 'data') & (df_dre_ultimo_mes.columns != 'lucro_por_acao')]:
                df_dre_ultimo_mes[col] = df_dre_ultimo_mes[col].apply(converter_valor)

        df_dre_ultimo_mes['tic'] = acao

        lista_dataframes.append(df_dre_ultimo_mes)

        iteracao = iteracao+1

        if iteracao % 15 == 0:
            pd.concat(lista_dataframes).to_csv('dre_trimestral.csv')

        end_time = time.time()

        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time

        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60

        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")               